# import

In [1]:
import os
import glob
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.model_selection import KFold
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 300)

In [2]:
# data directory
data_dir = '../input/optiver-realized-volatility-prediction/'

# Recap (data format)

In [9]:
pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv').head()

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [10]:
pd.read_parquet('../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0').head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100
1,5,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100
2,5,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100
3,5,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100
4,5,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100


In [11]:
pd.read_parquet('../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0').head()

,time_id,seconds_in_bucket,price,size,order_count
0,5,21,1.002301,326,12
1,5,46,1.002778,128,4
2,5,50,1.002818,55,1
3,5,57,1.003155,121,5
4,5,68,1.003646,4,1


# WAP

In [16]:
# Function to calculate first WAP
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

# Function to calculate second WAP
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

# Log return, volatility

In [3]:
# Function to calculate the log of the return
# Remember that logb(x / y) = logb(x) - logb(y)
def log_return(series):
    return np.log(series).diff()

# Calculate the realized volatility
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

In [4]:
# Function to count unique elements of a series
def count_unique(series):
    return len(np.unique(series))

In [5]:
# Function to read our base train and test set
def read_train_test():
    train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
    test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
    # Create a key to merge with book and trade data
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    print(f'Our training set has {train.shape[0]} rows')
    return train, test

# Preprocessor (offer book)

まず、offer book に対しての前処理を行う。オリジナルの `book_train.parquet` には以下のカラムが含まれていないので、それを新たに定義する：

- wap
- log_return 
- wap_balance : $\text{WAP}_1-\text{WAP}_2$
- price_spread : $(p_{a1}-p_{b1})/(p_{a1}+p_{b1})/2$
- bid(ask)_spread : $p_1 - p_2$
- total_volume : $(n_{a1}+n_{a2})+(n_{b1}+n_{b2})$
- volume_imbalance : $|(n_{a1}+n_{a2})-(n_{b1}+n_{b2})|$

In [69]:
# Function to preprocess book data (for each stock id)
def book_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    # Calculate Wap
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    # Calculate log returns
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    # Calculate wap balance
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    # Calculate spread
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    
    # Dict for aggregations
    create_feature_dict = {
        'wap1': [np.sum, np.mean, np.std],
        'wap2': [np.sum, np.mean, np.std],
        'log_return1': [np.sum, realized_volatility, np.mean, np.std],
        'log_return2': [np.sum, realized_volatility, np.mean, np.std],
        'wap_balance': [np.sum, np.mean, np.std],
        'price_spread':[np.sum, np.mean, np.std],
        'bid_spread':[np.sum, np.mean, np.std],
        'ask_spread':[np.sum, np.mean, np.std],
        'total_volume':[np.sum, np.mean, np.std],
        'volume_imbalance':[np.sum, np.mean, np.std]
    }
    
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_feature_450 = get_stats_window(seconds_in_bucket = 450, add_suffix = True)
    df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
    df_feature_150 = get_stats_window(seconds_in_bucket = 150, add_suffix = True)
    
    # Merge all
    df_feature = df_feature.merge(df_feature_450, how = 'left', left_on = 'time_id_', right_on = 'time_id__450')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_150, how = 'left', left_on = 'time_id_', right_on = 'time_id__150')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__450', 'time_id__300', 'time_id__150'], axis = 1, inplace = True)
    
    # Create row_id so we can merge
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    return df_feature

## get_stats_window について

新たに定義した特徴量を持った offer book (ここでは `df_feature`）を、さらに時間で区分分けを行う（0, 0\~150, 150\~300, 300\~450, 450\~）。

In [59]:
df = pd.read_parquet('../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0')
# Calculate Wap
df['wap1'] = calc_wap1(df)
df['wap2'] = calc_wap2(df)
# Calculate log returns
df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
# Calculate wap balance
df['wap_balance'] = abs(df['wap1'] - df['wap2'])
# Calculate spread
df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
df['bid_spread'] = df['bid_price1'] - df['bid_price2']
df['ask_spread'] = df['ask_price1'] - df['ask_price2']
df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))

# Dict for aggregations
create_feature_dict = {
    'wap1': [np.sum, np.mean, np.std],
    'wap2': [np.sum, np.mean, np.std],
    'log_return1': [np.sum, realized_volatility, np.mean, np.std],
    'log_return2': [np.sum, realized_volatility, np.mean, np.std],
    'wap_balance': [np.sum, np.mean, np.std],
    'price_spread':[np.sum, np.mean, np.std],
    'bid_spread':[np.sum, np.mean, np.std],
    'ask_spread':[np.sum, np.mean, np.std],
    'total_volume':[np.sum, np.mean, np.std],
    'volume_imbalance':[np.sum, np.mean, np.std]
}

# Function to get group stats for different windows (seconds in bucket)
def get_stats_window(seconds_in_bucket, add_suffix = False):
    # Group by the window
    df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
    # Rename columns joining suffix
    df_feature.columns = ['_'.join(col) for col in df_feature.columns]
    # Add a suffix to differentiate windows
    if add_suffix:
        df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
    return df_feature

### 各データについて

生データは、time_id と seconds_in_bucket が独立している。そこで、`get_stats_window` 以下の流れを作っている

- `groupby('time_id')` : 同じtime window のものをまとめる
    - 元データは (917553,20) --> (3830, 33) になる
    - つまり 3830 のユニークな time_id があるということ
- カラム名は seconds_in_bucket 毎に suffix をつけているので区別できる。
- あとは left join をしていく（キーをどうするかは `left_on, right_on`で指定している）
    - 元データは (3830, 33) --> (3830, 33 $\times$ 結合回数)になる
- 最終的に time_id から row_id を作成する

In [60]:
print(df.shape)
df.head()

(917553, 20)


,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,wap1,wap2,log_return1,log_return2,wap_balance,price_spread,bid_spread,ask_spread,total_volume,volume_imbalance
0,5,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100,1.001434,1.001390,NaN,NaN,0.000044,0.000878,0.000052,-0.000052,331,321
1,5,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100,1.001448,1.001390,0.000014,0.000000,0.000058,0.000878,0.000052,-0.000052,205,195
2,5,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100,1.001448,1.001391,0.000000,0.000001,0.000057,0.000878,0.000052,-0.000103,205,195
3,5,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100,1.001443,1.001391,-0.000005,0.000000,0.000052,0.000878,0.000052,-0.000103,231,221
4,5,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100,1.001443,1.001391,0.000000,0.000000,0.000052,0.000878,0.000052,-0.000103,231,221


In [51]:
df_feature     = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
df_feature_450 = get_stats_window(seconds_in_bucket = 450, add_suffix = True)

In [56]:
print(df_feature.shape)
df_feature.head()

(3830, 33)


,time_id_,wap1_sum,wap1_mean,wap1_std,wap2_sum,wap2_mean,wap2_std,log_return1_sum,log_return1_realized_volatility,log_return1_mean,log_return1_std,log_return2_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,wap_balance_sum,wap_balance_mean,wap_balance_std,price_spread_sum,price_spread_mean,price_spread_std,bid_spread_sum,bid_spread_mean,bid_spread_std,ask_spread_sum,ask_spread_mean,ask_spread_std,total_volume_sum,total_volume_mean,total_volume_std,volume_imbalance_sum,volume_imbalance_mean,volume_imbalance_std
0,5,303.125061,1.003725,0.000693,303.105539,1.003661,0.000781,0.002292,0.004499,7.613599e-06,0.000260,0.002325,0.006999,0.000008,0.000404,0.117051,0.000388,0.000295,0.257255,0.000852,0.000211,0.053006,0.000176,0.000162,-0.045557,-0.000151,0.000126,97696,323.496689,138.101214,40738,134.894040,107.260583
1,11,200.047768,1.000239,0.000262,200.041171,1.000206,0.000272,0.000360,0.001204,1.810239e-06,0.000086,0.000801,0.002476,0.000004,0.000176,0.042312,0.000212,0.000155,0.078836,0.000394,0.000157,0.028358,0.000142,0.000148,-0.027001,-0.000135,0.000065,82290,411.450000,172.263581,28410,142.050000,102.139758
2,16,187.913849,0.999542,0.000864,187.939824,0.999680,0.000862,-0.002074,0.002369,-1.109201e-05,0.000173,-0.001493,0.004801,-0.000008,0.000352,0.062228,0.000331,0.000246,0.136330,0.000725,0.000164,0.036955,0.000197,0.000170,-0.037243,-0.000198,0.000171,78274,416.351064,138.433034,26586,141.414894,108.891243
3,31,119.859781,0.998832,0.000757,119.835941,0.998633,0.000656,-0.002828,0.002574,-2.376661e-05,0.000236,-0.002053,0.003637,-0.000017,0.000334,0.045611,0.000380,0.000248,0.103252,0.000860,0.000280,0.022764,0.000190,0.000199,-0.013001,-0.000108,0.000091,52232,435.266667,156.120334,17546,146.216667,121.533215
4,62,175.932865,0.999619,0.000258,175.934256,0.999626,0.000317,-0.000002,0.001894,-1.057099e-08,0.000144,-0.000281,0.003257,-0.000002,0.000247,0.044783,0.000254,0.000188,0.069901,0.000397,0.000130,0.033565,0.000191,0.000083,-0.019206,-0.000109,0.000076,60407,343.221591,158.054066,21797,123.846591,102.407501


In [58]:
print(df_feature_450.shape)
df_feature_450.head()

(3830, 33)


,time_id__450,wap1_sum_450,wap1_mean_450,wap1_std_450,wap2_sum_450,wap2_mean_450,wap2_std_450,log_return1_sum_450,log_return1_realized_volatility_450,log_return1_mean_450,log_return1_std_450,log_return2_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,wap_balance_sum_450,wap_balance_mean_450,wap_balance_std_450,price_spread_sum_450,price_spread_mean_450,price_spread_std_450,bid_spread_sum_450,bid_spread_mean_450,bid_spread_std_450,ask_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,total_volume_sum_450,total_volume_mean_450,total_volume_std_450,volume_imbalance_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450
0,5,68.236749,1.003482,0.000514,68.231672,1.003407,0.000640,-0.000361,0.001721,-0.000005,0.000210,0.000068,0.004114,0.000001,0.000503,0.024868,0.000366,0.000277,0.053236,0.000783,0.000181,0.017790,0.000262,0.000178,-0.011274,-0.000166,0.000126,17948,263.941176,116.940077,9620,141.470588,84.467864
1,11,54.027991,1.000518,0.000235,54.021532,1.000399,0.000287,-0.000059,0.000918,-0.000001,0.000126,0.000488,0.001883,0.000009,0.000258,0.014524,0.000269,0.000175,0.018812,0.000348,0.000144,0.012598,0.000233,0.000239,-0.007729,-0.000143,0.000066,24191,447.981481,177.264272,5275,97.685185,88.144569
2,16,43.922425,0.998237,0.000541,43.933158,0.998481,0.000766,-0.001469,0.001158,-0.000033,0.000173,-0.001831,0.002972,-0.000042,0.000451,0.016055,0.000365,0.000282,0.026608,0.000605,0.000105,0.008186,0.000186,0.000217,-0.009143,-0.000208,0.000168,20201,459.113636,116.212559,6869,156.113636,102.024670
3,31,17.965415,0.998079,0.000430,17.971630,0.998424,0.000544,-0.000526,0.000993,-0.000029,0.000239,-0.000882,0.001424,-0.000049,0.000342,0.006441,0.000358,0.000253,0.019047,0.001058,0.000074,0.002082,0.000116,0.000046,-0.000879,-0.000049,0.000011,9720,540.000000,153.413704,2628,146.000000,106.693624
4,62,35.982653,0.999518,0.000257,35.991844,0.999773,0.000212,0.000397,0.001378,0.000011,0.000233,-0.000298,0.000966,-0.000008,0.000163,0.013087,0.000364,0.000203,0.018700,0.000519,0.000138,0.007040,0.000196,0.000024,-0.004895,-0.000136,0.000066,14110,391.944444,123.180227,4212,117.000000,99.328028


In [61]:
print(df_feature.merge(df_feature_450, how = 'left', left_on = 'time_id_', right_on = 'time_id__450').shape)
df_feature.merge(df_feature_450, how = 'left', left_on = 'time_id_', right_on = 'time_id__450').head()

(3830, 66)


,time_id_,wap1_sum,wap1_mean,wap1_std,wap2_sum,wap2_mean,wap2_std,log_return1_sum,log_return1_realized_volatility,log_return1_mean,log_return1_std,log_return2_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,wap_balance_sum,wap_balance_mean,wap_balance_std,price_spread_sum,price_spread_mean,price_spread_std,bid_spread_sum,bid_spread_mean,bid_spread_std,ask_spread_sum,ask_spread_mean,ask_spread_std,total_volume_sum,total_volume_mean,total_volume_std,volume_imbalance_sum,volume_imbalance_mean,volume_imbalance_std,time_id__450,wap1_sum_450,wap1_mean_450,wap1_std_450,wap2_sum_450,wap2_mean_450,wap2_std_450,log_return1_sum_450,log_return1_realized_volatility_450,log_return1_mean_450,log_return1_std_450,log_return2_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,wap_balance_sum_450,wap_balance_mean_450,wap_balance_std_450,price_spread_sum_450,price_spread_mean_450,price_spread_std_450,bid_spread_sum_450,bid_spread_mean_450,bid_spread_std_450,ask_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,total_volume_sum_450,total_volume_mean_450,total_volume_std_450,volume_imbalance_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450
0,5,303.125061,1.003725,0.000693,303.105539,1.003661,0.000781,0.002292,0.004499,7.613599e-06,0.000260,0.002325,0.006999,0.000008,0.000404,0.117051,0.000388,0.000295,0.257255,0.000852,0.000211,0.053006,0.000176,0.000162,-0.045557,-0.000151,0.000126,97696,323.496689,138.101214,40738,134.894040,107.260583,5,68.236749,1.003482,0.000514,68.231672,1.003407,0.000640,-0.000361,0.001721,-0.000005,0.000210,0.000068,0.004114,0.000001,0.000503,0.024868,0.000366,0.000277,0.053236,0.000783,0.000181,0.017790,0.000262,0.000178,-0.011274,-0.000166,0.000126,17948,263.941176,116.940077,9620,141.470588,84.467864
1,11,200.047768,1.000239,0.000262,200.041171,1.000206,0.000272,0.000360,0.001204,1.810239e-06,0.000086,0.000801,0.002476,0.000004,0.000176,0.042312,0.000212,0.000155,0.078836,0.000394,0.000157,0.028358,0.000142,0.000148,-0.027001,-0.000135,0.000065,82290,411.450000,172.263581,28410,142.050000,102.139758,11,54.027991,1.000518,0.000235,54.021532,1.000399,0.000287,-0.000059,0.000918,-0.000001,0.000126,0.000488,0.001883,0.000009,0.000258,0.014524,0.000269,0.000175,0.018812,0.000348,0.000144,0.012598,0.000233,0.000239,-0.007729,-0.000143,0.000066,24191,447.981481,177.264272,5275,97.685185,88.144569
2,16,187.913849,0.999542,0.000864,187.939824,0.999680,0.000862,-0.002074,0.002369,-1.109201e-05,0.000173,-0.001493,0.004801,-0.000008,0.000352,0.062228,0.000331,0.000246,0.136330,0.000725,0.000164,0.036955,0.000197,0.000170,-0.037243,-0.000198,0.000171,78274,416.351064,138.433034,26586,141.414894,108.891243,16,43.922425,0.998237,0.000541,43.933158,0.998481,0.000766,-0.001469,0.001158,-0.000033,0.000173,-0.001831,0.002972,-0.000042,0.000451,0.016055,0.000365,0.000282,0.026608,0.000605,0.000105,0.008186,0.000186,0.000217,-0.009143,-0.000208,0.000168,20201,459.113636,116.212559,6869,156.113636,102.024670
3,31,119.859781,0.998832,0.000757,119.835941,0.998633,0.000656,-0.002828,0.002574,-2.376661e-05,0.000236,-0.002053,0.003637,-0.000017,0.000334,0.045611,0.000380,0.000248,0.103252,0.000860,0.000280,0.022764,0.000190,0.000199,-0.013001,-0.000108,0.000091,52232,435.266667,156.120334,17546,146.216667,121.533215,31,17.965415,0.998079,0.000430,17.971630,0.998424,0.000544,-0.000526,0.000993,-0.000029,0.000239,-0.000882,0.001424,-0.000049,0.000342,0.006441,0.000358,0.000253,0.019047,0.001058,0.000074,0.002082,0.000116,0.000046,-0.000879,-0.000049,0.000011,9720,540.000000,153.413704,2628,146.000000,106.693624
4,62,175.932865,0.999619,0.000258,175.934256,0.999626,0.000317,-0.000002,0.001894,-1.057099e-08,0.000144,-0.000281,0.003257,-0.000002,0.000247,0.044783,0.000254,0.000188,0.069901,0.000397,0.000130,0.033565,0.000191,0.000083,-0.019206,-0.000109,0.000076,60407,343.221591,158.054066,21797,123.846591,102.407501,62,35.982653,0.999518,0.000257,35.991844,0.999773,0.000212,0.000397,

## pandas.aggregateについて

`agg` or `aggregate`（同名関数）を用いることで、様々な処理を一括で行うことができる。

In [29]:
tmp = pd.DataFrame({"a":[1,2,3,4,5], "b":[6,7,8,9,10], "c":[11,12,13,14,15]})
tmp.agg([np.sum, np.mean])

,a,b,c
sum,15.0,40.0,65.0
mean,3.0,8.0,13.0


また辞書型として与えることで、任意の列に任意の処理を適用することができる。

In [39]:
tmp = pd.DataFrame({"a":[1,2,3,4,5], "b":[1,2,3,4,5], "c":[1,2,3,4,5]})
tmp.agg({"a":[np.mean, np.sum], "b":[np.mean, np.sum]})

,a,b
mean,3.0,3.0
sum,15.0,15.0


また、独自に関数を定義して適用することもできる。今回の処理で行っているのは、とある列で groupby して（`time_id`）、aggregate 処理を適用している。そのときに独自の関数も適用している。ただ、そのまま放置すると MultiIndex で返ってしまうので、以降の処理を簡易にするためにカラム名を上書きする必要がある（別にMultiIndexで扱っても良いけど）。また `reset_index` を挟むことで、 一旦groupbyでインデックスになったものをカラムに戻す。

In [48]:
def test1(series):
    return series+1
def test2(series):
    return series-1

tmp = pd.DataFrame({"a":[1,2,3,4,5], "b":[1,2,3,4,5], "c":[1,2,3,4,5]})
tmp = tmp.groupby("c").agg({"a":[np.mean, np.sum, test1], "b":[np.mean, np.sum, test2]})
tmp

a              b          
  mean sum test1 mean sum test2
c                              
1    1   1     2    1   1     0
2    2   2     3    2   2     1
3    3   3     4    3   3     2
4    4   4     5    4   4     3
5    5   5     6    5   5     4

In [49]:
tmp = tmp.reset_index()
tmp.columns = ['_'.join(col) for col in tmp.columns]
tmp

,c_,a_mean,a_sum,a_test1,b_mean,b_sum,b_test2
0,1,1,1,2,1,1,0
1,2,2,2,3,2,2,1
2,3,3,3,4,3,3,2
3,4,4,4,5,4,4,3
4,5,5,5,6,5,5,4


# Preprocessor (trade)

offer book の前処理でやったことを同じことを繰り返す。

In [70]:
# Function to preprocess trade data (for each stock id)
def trade_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    
    # Dict for aggregations
    create_feature_dict = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum],
        'order_count':[np.mean],
    }
    
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_feature_450 = get_stats_window(seconds_in_bucket = 450, add_suffix = True)
    df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
    df_feature_150 = get_stats_window(seconds_in_bucket = 150, add_suffix = True)

    # Merge all
    df_feature = df_feature.merge(df_feature_450, how = 'left', left_on = 'time_id_', right_on = 'time_id__450')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_150, how = 'left', left_on = 'time_id_', right_on = 'time_id__150')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__450', 'time_id__300', 'time_id__150'], axis = 1, inplace = True)
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    return df_feature

## 前処理の詳細

はじめに groupby して、log_return を追加しておく。

In [64]:
tmp = pd.read_parquet('../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0')
tmp['log_return'] = tmp.groupby('time_id')['price'].apply(log_return)
tmp

,time_id,seconds_in_bucket,price,size,order_count,log_return
0,5,21,1.002301,326,12,NaN
1,5,46,1.002778,128,4,0.000476
2,5,50,1.002818,55,1,0.000040
3,5,57,1.003155,121,5,0.000336
4,5,68,1.003646,4,1,0.000489
...,...,...,...,...,...,...
123438,32767,471,0.998659,200,3,0.000144
123439,32767,517,0.998515,90,1,-0.000144
123440,32767,523,0.998563,1,1,0.000048
123441,32767,542,0.998803,90,4,0.000240


# Preprocessor

In [72]:
# Funtion to make preprocessing function in parallel (for each stock id)
def preprocessor(list_stock_ids, is_train = True):
    
    # Parrallel for loop
    def for_joblib(stock_id):
        # Train
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        # Test
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
        # Preprocess book and trade data and merge them
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        
        # Return the merge dataframe
        return df_tmp
    
    # Use parallel api to call paralle for loop
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    # Concatenate all the dataframes that return from Parallel
    df = pd.concat(df, ignore_index = True)
    return df

## joblib について

joblibを用いることで、シンプルに並行処理を行うことができる。並行処理には `Parallel` と `delayed` を用いる。

```python
joblib.Parallel(<Parallelへの引数>)(
    joblib.delayed(<実行する関数>)(<関数への引数>) for 変数名 in イテラブル
)
```

ここでは、n_jobs = -1, verbose = 1 を並行処理の引数としている。また並行処理させる関数は `preprocessor` の内部で定義されている `for_joblib`である。この関数はストックID（株銘柄を識別するためのID）を引数に持ち、book と trade を左結合したDFを返す。Parallelの返り値はこれらのDFをID個数分であり、最終的にそれらを concat して縦方向に連結している（インデックスは無視）。

この preprocessor で作成した DF は、train.csv から作成したDFと結合される。

# get_time_stock

- （いまさらだが）groupby-->agg/apply で新規作成した列名は、その関数名がsuffixとして自動で付いている。なので見に覚えのない `log_return1_realized_volatility_...`という列名を持ったDFをいつの間にか扱っていることに留意する。

In [83]:
# Function to get group stats for the stock_id and time_id
def get_time_stock(df):
    # Get realized volatility columns
    vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_450', 'log_return2_realized_volatility_450', 
                'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_150', 'log_return2_realized_volatility_150', 
                'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_450', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_150']

    # Group by the stock id
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # Rename columns joining suffix
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    # Group by the stock id
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # Rename columns joining suffix
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    # Merge with original dataframe
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df

# Train 前のデータ最終形態

In [87]:
# Read train.csv and test.csv
train, test = read_train_test()

# Get unique stock ids 
train_stock_ids = [0,]

# Preprocess them using Parallel and our single stock id functions
train_ = preprocessor(train_stock_ids, is_train = True)
train = train.merge(train_, on = ['row_id'], how = 'left')

# Get group stats of time_id and stock_id
train = get_time_stock(train)

train[train["stock_id"]==0]

Our training set has 428932 rows


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   25.0s finished


,stock_id,time_id,target,row_id,wap1_sum,wap1_mean,wap1_std,wap2_sum,wap2_mean,wap2_std,log_return1_sum,log_return1_realized_volatility,log_return1_mean,log_return1_std,log_return2_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,wap_balance_sum,wap_balance_mean,wap_balance_std,price_spread_sum,price_spread_mean,price_spread_std,bid_spread_sum,bid_spread_mean,bid_spread_std,ask_spread_sum,ask_spread_mean,ask_spread_std,total_volume_sum,total_volume_mean,total_volume_std,volume_imbalance_sum,volume_imbalance_mean,volume_imbalance_std,wap1_sum_450,wap1_mean_450,wap1_std_450,wap2_sum_450,wap2_mean_450,wap2_std_450,log_return1_sum_450,log_return1_realized_volatility_450,log_return1_mean_450,log_return1_std_450,log_return2_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,wap_balance_sum_450,wap_balance_mean_450,wap_balance_std_450,price_spread_sum_450,price_spread_mean_450,price_spread_std_450,bid_spread_sum_450,bid_spread_mean_450,bid_spread_std_450,ask_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,total_volume_sum_450,total_volume_mean_450,total_volume_std_450,volume_imbalance_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450,wap1_sum_300,wap1_mean_300,wap1_std_300,wap2_sum_300,wap2_mean_300,wap2_std_300,log_return1_sum_300,log_return1_realized_volatility_300,log_return1_mean_300,log_return1_std_300,log_return2_sum_300,log_return2_realized_volatility_300,log_return2_mean_300,log_return2_std_300,wap_balance_sum_300,wap_balance_mean_300,wap_balance_std_300,price_spread_sum_300,price_spread_mean_300,price_spread_std_300,bid_spread_sum_300,bid_spread_mean_300,bid_spread_std_300,ask_spread_sum_300,ask_spread_mean_300,ask_spread_std_300,total_volume_sum_300,total_volume_mean_300,total_volume_std_300,volume_imbalance_sum_300,volume_imbalance_mean_300,volume_imbalance_std_300,wap1_sum_150,wap1_mean_150,wap1_std_150,wap2_sum_150,wap2_mean_150,wap2_std_150,log_return1_sum_150,log_return1_realized_volatility_150,log_return1_mean_150,log_return1_std_150,log_return2_sum_150,log_return2_realized_volatility_150,log_return2_mean_150,log_return2_std_150,wap_balance_sum_150,wap_balance_mean_150,wap_balance_std_150,price_spread_sum_150,price_spread_mean_150,price_spread_std_150,bid_spread_sum_150,bid_spread_mean_150,bid_spread_std_150,ask_spread_sum_150,ask_spread_mean_150,ask_spread_std_150,total_volume_sum_150,total_volume_mean_150,total_volume_std_150,volume_imbalance_sum_150,volume_imbalance_mean_150,volume_imbalance_std_150,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_450,trade_seconds_in_bucket_count_unique_450,trade_size_sum_450,trade_order_count_mean_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_mean_150,log_return1_realized_volatility_mean_stock,log_return1_realized_volatility_std_stock,log_return1_realized_volatility_max_stock,log_return1_realized_volatility_min_stock,log_return2_realized_volatility_mean_stock,log_return2_realized_volatility_std_stock,log_return2_realized_volatility_max_stock,log_return2_realized_volatility_min_stock,log_return1_realized_volatility_450_mean_stock,log_return1_realized_volatility_450_std_stock,log_return1_realized_volatility_450_max_stock,log_return1_realized_volatility_450_min_stock,log_return2_realized_volatility_450_mean_stock,log_return2_realized_volatility_450_std_stock,log_return2_realized_volatility_450_max_stock,log_return2_realized_volatility_450_min_stock,log_return1_realized_volatility_300_mean_stock,log_return1_realized_volatility_300_std_stock,log_return1_realized_volatility_300_max_stock,log_return1_realized_volatility_300_min_stock,log_return2_realized_volatility_300_mean_stock,log_return2_realized_volatility_300_std_

## 結局やっていること

### book_data
1. WAP, log_return, WAP/bid/ask/size の残差 を計算
1. データをseconds_in_bucketで分割する（0~, 150~, 300~, 450~）
1. それらの情報の sum, mean, std, realized_volatility を、time_id (=time windowのこと) 毎に groupbyして計算する

### trade

1. log_return を計算する
1. データをseconds_in_bucketで分割する（book_data と違ってスパースな時間区分であることに留意）
1. それらの情報を realized_volatility, count_unique, np.sum, mean で計算する

### train

1. 以上で計算した book・trade を stock_id 毎に用意して、それを縦連結する
1. 時間で分割、stock_id でループを回しているのでややこしく見えるが大したことはやってない

# train (LightGBM)

In [66]:
# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

In [67]:
def train_and_evaluate(train, test):
    # Hyperparammeters (just basic)
    params = {
      'objective': 'rmse',  
      'boosting_type': 'gbdt',
      'num_leaves': 100,
      'n_jobs': -1,
      'learning_rate': 0.1,
      'feature_fraction': 0.8,
      'bagging_fraction': 0.8,
      'verbose': -1
    }
    
    # Split features and target
    x = train.drop(['row_id', 'target', 'time_id'], axis = 1)
    y = train['target']
    x_test = test.drop(['row_id', 'time_id'], axis = 1)
    # Transform stock id to a numeric value
    x['stock_id'] = x['stock_id'].astype(int)
    x_test['stock_id'] = x_test['stock_id'].astype(int)
    
    # Create out of folds array
    oof_predictions = np.zeros(x.shape[0])
    # Create test array to store predictions
    test_predictions = np.zeros(x_test.shape[0])
    # Create a KFold object
    kfold = KFold(n_splits = 5, random_state = 66, shuffle = True)
    # Iterate through each fold
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(x)):
        print(f'Training fold {fold + 1}')
        x_train, x_val = x.iloc[trn_ind], x.iloc[val_ind]
        y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
        # Root mean squared percentage error weights
        train_weights = 1 / np.square(y_train)
        val_weights = 1 / np.square(y_val)
        train_dataset = lgb.Dataset(x_train, y_train, weight = train_weights, categorical_feature = ['stock_id'])
        val_dataset = lgb.Dataset(x_val, y_val, weight = val_weights, categorical_feature = ['stock_id'])
        model = lgb.train(params = params, 
                          train_set = train_dataset, 
                          valid_sets = [train_dataset, val_dataset], 
                          num_boost_round = 10000, 
                          early_stopping_rounds = 50, 
                          verbose_eval = 50,
                          feval = feval_rmspe)
        # Add predictions to the out of folds array
        oof_predictions[val_ind] = model.predict(x_val)
        # Predict the test set
        test_predictions += model.predict(x_test) / 5
        
    rmspe_score = rmspe(y, oof_predictions)
    print(f'Our out of folds RMSPE is {rmspe_score}')
    # Return test predictions
    return test_predictions

# 実際にLightGBMを回してみる

In [89]:
# Read train.csv and test.csv
train, test = read_train_test()

# Get unique stock ids 
train_stock_ids = train['stock_id'].unique()
# Set 0 for local macbook PC
train_stock_ids = [0, ]

# Preprocess them using Parallel and our single stock id functions
train_ = preprocessor(train_stock_ids, is_train = True)
train = train.merge(train_, on = ['row_id'], how = 'left')

# Get unique stock ids 
test_stock_ids = test['stock_id'].unique()
# Preprocess them using Parallel and our single stock id functions
test_ = preprocessor(test_stock_ids, is_train = False)
test = test.merge(test_, on = ['row_id'], how = 'left')

# Get group stats of time_id and stock_id
train = get_time_stock(train)
test = get_time_stock(test)

Our training set has 428932 rows


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   25.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.9s finished


In [90]:
# Traing and evaluate
test_predictions = train_and_evaluate(train, test)
# Save test predictions
test['target'] = test_predictions
#test[['row_id', 'target']].to_csv('submission.csv',index = False)

Training fold 1
Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 0.00060594	training's RMSPE: 0.280645	valid_1's rmse: 0.000624408	valid_1's RMSPE: 0.288123
[100]	training's rmse: 0.000553776	training's RMSPE: 0.256485	valid_1's rmse: 0.000584433	valid_1's RMSPE: 0.269677
[150]	training's rmse: 0.000528296	training's RMSPE: 0.244684	valid_1's rmse: 0.000568897	valid_1's RMSPE: 0.262509
[200]	training's rmse: 0.000512281	training's RMSPE: 0.237267	valid_1's rmse: 0.000561137	valid_1's RMSPE: 0.258928
[250]	training's rmse: 0.000500215	training's RMSPE: 0.231678	valid_1's rmse: 0.000555248	valid_1's RMSPE: 0.256211
[300]	training's rmse: 0.000490539	training's RMSPE: 0.227196	valid_1's rmse: 0.0005509	valid_1's RMSPE: 0.254204
[350]	training's rmse: 0.000482713	training's RMSPE: 0.223572	valid_1's rmse: 0.000547611	valid_1's RMSPE: 0.252686
[400]	training's rmse: 0.000476017	training's RMSPE: 0.22047	valid_1's rmse: 0.000545258	valid_1's RMSPE: 0.251601
